In [48]:
using DataFrames
using DataFramesMeta
using GLM
using CSV

In [46]:
# figure out how to read in types
#data = vec(readdlm("coltypes.csv", ',', String))
function make_dataset(file,types,label,case,control,X)
    df = CSV.read(file, DataFrame; delim='\t', header=true, types=types)
    @subset!(df, $label  .== case .|| $label .== control)
    df = df[:, vcat(X,label)]
    return @transform!(df, $label = ($label .== case))
end

make_dataset (generic function with 1 method)

In [49]:
function fit_logit(df,label)
    X = sum(Term.(Symbol.(names(df[:, Not(label)]))))
    logit = glm(Term(label) ~ X, df, Binomial(), LogitLink())
    return logit
end

fit_logit (generic function with 1 method)

In [52]:
types=[String,String,String,String]
df = make_dataset("example_data/titanic.txt",types,"Survived","Yes","No",["Class"])
logit = fit_logit(df,:Survived)
predict(logit)

2201-element Vector{Float64}:
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 0.2521246458929164
 ⋮
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121
 0.23954802259934121